## Dataset. model_dataset
<br>
<img src="./images/model_dataset.png"/>

Part explanation for the columns:  
1. label_group (obeject): 4 groups of resonse to offers
    - 'none_offer'
    - 'no_care'
    - 'tried'
    - 'effective_offer'
2. label_seg (int): 12 segments based on age and gender
    - values: 1 ... 12  <br>  
  
(More details in 2_heuristic_exploration.ipynb)

###  <u>10 Kinds</u> of offer_id
| offer_id #| type | duration | requirement | reward |
|:-| :-| :-:|:-:|:-:|
| 0 | bogo | 7 | 10 | 10 |
| 1 | bogo | 5 | 10 | 10 |
| 2 | infomational | 4 | - | - |
| 3 | bogo | 7 | 5 | 5 |
| 4 | discount | 10 | 20 | 5 |
| 5 | discount | 7 | 7 | 3 |
| 6 | discount | 10 | 10 | 2 |
| 7 | informational | 3 | - | - |
| 8 | bogo | 5 | 5 | 5 |
| 9 | discount | 7 | 10 | 2 |

### <u>12 Segements</u> based on 'age' and 'gender'
<br>
    
|Segment #| Age Group (edge included)<br> (Experiment in 2018) | Gender | 
|---| --- | --- | 
|1| Millenials(-21 & 22-37) | M | 
|2| Millenials(-21 & 22-37) | F | 
|3| Millenials(-21 & 22-37) | O | 
|4| Gen X(38-53) | M|
|5| Gen X(38-53) | F|
|6| Gen X(38-53) | O|
|7| Baby Boomer(54-72) | M|
|8| Baby Boomer(54-72) | F|
|9| Baby Boomer(54-72) | O|
|10| Silent(73-90 & 91+) | M|
|11| Silent(73-90 & 91+) | F|
|12| Silent(73-90 & 91+) | O|

### <u>4 Groups</u> of possible responsiveness to offer
<br>

|Group| received | viewed |valid completed | transaction amount |Scenario |
| :-| :-: | :-:| :-: | :-: | :- |
|1.none_offer| 0 | 0 | 0 | |haven't received the offer |
|2.no_care | 1 | 0 | - | |received but not viewed.<br> regarded as no_care|
|| 1 | 1 | 0 | =0.0 | received, viewed but no transaction |
|| 1 | 1 | 1<br>viewed after completed |  | received, but completed unintentionally |
|3.tried| 1 | 1 | 0 | >0.0|received, viewed, have transaction |
|4.effctive_offer | 1 | 1 | 1<br>viewed before completed | | viewed before completed,  effctive offer|

# <a class="anchor" id="Start">Table of Contents</a>

I. [Feature Engineer](#1)<br>
II.[Segement of the Person](#2)<br>
III.[Classes of responsiveness to the offers](#3)<br>
   1. [Divide the response into 4 groups](#3.1)<br>
   2. [CHECK the dividing method for the 4 groups of response to offer](#3.2)<br>
   3. [Heuristic exploration in 4 aspects](#3.3)<br>
    - 3.1 [Offer Distributions: none_offer VS. others](#3.3.1)<br>
        - Conclusion<br>
    - 3.2 [Interest Distributions: no_care VS. (tried & effective_offer)](#3.3.2)<br>
        - Conclusion<br>
    - 3.3 [Difficulty Distributions: tried VS. effective_offer](#3.3.3)<br>
        - Conclusion<br>
    - 3.4 [Index NIR: Is the offer significant popular?](#3.3.4)<br>
        - Conclusion<br>
    
IV.[Save the dataset for ML modeling](#4)<br>
[References](#References)

In [4]:
import pandas as pd
import numpy as np
import math
import json

from time import time
from datetime import date
from collections import defaultdict

import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

model_dataset_raw = pd.read_csv('./data_generated/model_raw_dataset.csv', dtype={'offer_id': str})

In [6]:
model_dataset_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66506 entries, 0 to 66505
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   person                 66506 non-null  int64  
 1   offer_id               66506 non-null  object 
 2   time_received          66501 non-null  float64
 3   time_viewed            49860 non-null  float64
 4   time_transaction       57752 non-null  object 
 5   time_completed         40407 non-null  float64
 6   amount_with_offer      66506 non-null  float64
 7   label_effective_offer  66506 non-null  int64  
 8   reward                 66501 non-null  float64
 9   difficulty             66501 non-null  float64
 10  duration               66501 non-null  float64
 11  offer_type             66501 non-null  object 
 12  email                  66501 non-null  float64
 13  mobile                 66501 non-null  float64
 14  social                 66501 non-null  float64
 15  we

## <a class="anchor" id="1">[I. Feature Engineer](#Start)</a>

## <a class="anchor" id="References">[References](#Start)</a>
[[1]Starbucks Capstone Challenge: Using Starbucks app user data to predict effective offers](https://github.com/syuenloh/UdacityDataScientistCapstone/blob/master/Starbucks%20Capstone%20Challenge%20-%20Using%20Starbucks%20app%20user%20data%20to%20predict%20effective%20offers.ipynb)<br>
